<a href="https://colab.research.google.com/github/chaw-thiri/DatabaseCourse/blob/main/w12_timeseries_lab_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import sqlite3 as sl
con = sl.connect('my-test.db')

In [28]:
with con:
    con.execute("""
    CREATE TABLE USER(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        age INTEGER
    );
    """)

In [29]:
sql = 'INSERT INTO USER(id,name,age) values(?,?,?)'
data =[
    (1,'Alice',21),
    (2,'Bob',22),
    (3,'Chris',23)
]

In [30]:
with con:
    con.executemany(sql,data)

In [31]:
with con:
    data = con.execute("SELECT * FROM USER WHERE AGE <= 22")
    for row in data:
        print(row)

(1, 'Alice', 21)
(2, 'Bob', 22)


In [32]:
# integrate sqlite with pandas
import pandas as pd
df_skill = pd.DataFrame({
    'user_id':[1,1,2,2,2,3,3],
    'skill':['Network Security','Algorithm Development','Network Security','Java','Python','Data Science','Machine Learning']
})
df_skill

,user_id,skill
0,1,Network Security
1,1,Algorithm Development
2,2,Network Security
3,2,Java
4,2,Python
5,3,Data Science
6,3,Machine Learning


In [33]:
df_skill.to_sql('SKILL',con)
df = pd.read_sql('''
SELECT s.user_id, u.name, u.age, s.skill
    FROM USER u
    LEFT JOIN SKILL s ON u.id = s.user_id
''',con)
df

,user_id,name,age,skill
0,1,Alice,21,Algorithm Development
1,1,Alice,21,Network Security
2,2,Bob,22,Java
3,2,Bob,22,Network Security
4,2,Bob,22,Python
5,3,Chris,23,Data Science
6,3,Chris,23,Machine Learning


In [34]:
df.to_sql('USER_SKILL',con)

7

In [41]:
with con:
    data = con.execute("SELECT name FROM sqlite_schema WHERE type = 'table' AND name NOT LIKE 'sqlite_%'")
    for row in data:
        print(row)

('USER',)
('SKILL',)
('USER_SKILL',)


USING SQLITE AS A TIMESERIES DATABASE

In [35]:
from random import uniform,choice
import time
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline

In [37]:
#compute the current day
import sqlite3

conn = sqlite3.connect('AdvancedSQLLab3.db')

cursor = conn.cursor()
r_list = cursor.execute("SELECT date('now')")
for rows in r_list:
    print(rows)

conn.commit()
conn.close()

('2023-11-13',)


In [38]:
#compute the last day of the current month
import sqlite3

conn = sqlite3.connect('AdvancedSQLLab3.db')

cursor = conn.cursor()
r_list = cursor.execute("SELECT date('now','start of month','+1 month','-1 day')")
for rows in r_list:
    print(rows)

conn.commit()
conn.close()

('2023-11-30',)


Inserting 500 rows of temperature which ranges from 18 to 26

In [39]:
import sqlite3

conn = sqlite3.connect('AdvancedSQLLab3.db')

cursor = conn.cursor()

# create index : for efficient querying , we'll want to index the time stamp column
commands =[
    """
    CREATE TABLE Temperature(Timestamp DATETIME NOT NULL, Temperature NUMERIC NOT NULL);
    """,
    """
    CREATE UNIQUE INDEX idx_timestamp ON Temperature(Timestamp);

    """
]
for command in commands:
    cursor.execute(command)


def dt(days):
    return timedelta(days = days)

N_rows = 500
now= datetime.now()


for i in range(N_rows):
    timestamp = now -dt(days=(N_rows-i))
    temperature = uniform(18,26)
    cursor.execute("INSERT INTO Temperature VALUES (:timestamp, :temperature)",
                   {'timestamp':timestamp,'temperature':temperature})

r_list= cursor.execute("SELECT datetime(Timestamp) as Date, Temperature from Temperature")
for rows in r_list:
    print(rows)
conn.commit()
conn.close()

('2022-07-01 00:45:18', 22.927460485536884)
('2022-07-02 00:45:18', 23.028351588675847)
('2022-07-03 00:45:18', 23.567208212477855)
('2022-07-04 00:45:18', 23.733393477260815)
('2022-07-05 00:45:18', 24.443956636534764)
('2022-07-06 00:45:18', 22.99117931366566)
('2022-07-07 00:45:18', 18.943851587316257)
('2022-07-08 00:45:18', 23.461799113423616)
('2022-07-09 00:45:18', 20.175822349043884)
('2022-07-10 00:45:18', 23.336369551181374)
('2022-07-11 00:45:18', 24.54287072076268)
('2022-07-12 00:45:18', 21.199649381642317)
('2022-07-13 00:45:18', 25.342646290699)
('2022-07-14 00:45:18', 19.201175414860842)
('2022-07-15 00:45:18', 18.95778166186016)
('2022-07-16 00:45:18', 19.685667221018633)
('2022-07-17 00:45:18', 18.077951384170028)
('2022-07-18 00:45:18', 18.625651349692845)
('2022-07-19 00:45:18', 24.626076483455204)
('2022-07-20 00:45:18', 24.64480490282243)
('2022-07-21 00:45:18', 24.440619959095386)
('2022-07-22 00:45:18', 19.569328731898512)
('2022-07-23 00:45:18', 21.419224485852